In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup, element, NavigableString, Tag
import mechanicalsoup
import re
import pandas as pd
import numpy as np


In [106]:
trials = pd.read_excel('../../data/failed_trials/Plana2022_suppl/PhaseIII_ClinicalTrials_metadata_F.xlsx', sheet_name='Trials')
criteria2 = pd.read_excel('../../data/failed_trials/major_cancer_types_search.xlsx', sheet_name='criteria 2')

In [108]:
trials = pd.read_excel('../../data/failed_trials/Plana2022_otherCancerTypes/PhaseIII_ClinicalTrials_metadata_OtherTumorTypes_DP.xlsx', sheet_name='Trials')
criteria2 = pd.read_excel('../../data/failed_trials/other_cancer_types_search.xlsx', sheet_name='criteria 2')

In [5]:
trials = pd.read_excel(
    '../../data/all_phase3_trials/all_phase3_trials_abstract.xlsx', sheet_name='Sheet1')


In [109]:
trial_names = criteria2['Trial Name_Figure'].str.split('_', expand=True)[0].unique()

In [110]:
trial_names.shape

(8,)

In [9]:
#pubmed_ids = trials[trials['Trial Name'].isin(trial_names)]['PubMed ID'].unique()
pubmed_ids = trials['PMID'].values


In [11]:
def clean_text(txt):
    return txt.strip().replace('  ', '').replace('\n\n', '\n').replace('\n\n', '\n')


In [16]:
def find_title_abstract(pubmed_id):
    url = f'https://pubmed.ncbi.nlm.nih.gov/{pubmed_id}/'
    page = urlopen(url)
    html = page.read().decode('utf-8')
    soup = BeautifulSoup(html, "html.parser")
    title = clean_text(soup.find("title").text)
    try:
        abstract = clean_text(
            soup.find("div", {"class": "abstract-content selected"}).text)
    except AttributeError:
        abstract = "NA"
    return (title, abstract)
    


In [15]:
criteria3 = pd.DataFrame(index=pubmed_ids, columns=['Title', 'Abstract'])

In [17]:
tmp = pd.Series(pubmed_ids).apply(find_title_abstract)

In [18]:
criteria3 = pd.DataFrame(tmp.to_list(), index=pubmed_ids, columns=['Title', 'Abstract'])
criteria3.index.name = 'PMID'

In [19]:
criteria3

,Title,Abstract
PMID,,
24256848,A randomized phase III trial comparing inducti...,Background:\nConcurrent chemoradiotherapy (CCR...
24297950,Randomized phase III trial of temsirolimus ver...,Purpose:\nThis international phase III trial (...
24297945,Randomized phase III trial of temsirolimus and...,Purpose:\nTo prospectively determine the effic...
24323035,"Randomized, placebo-controlled, phase III tria...",Purpose:\nWe evaluated angiogenesis-targeted s...
24356622,Final results from a randomized phase 3 study ...,Background:\nThe study 20050181 demonstrated s...
...,...,...
33207094,First-Line Lorlatinib or Crizotinib in Advance...,"Background:\nLorlatinib, a third-generation in..."
33278935,Pembrolizumab plus chemotherapy versus placebo...,Background:\nPembrolizumab monotherapy showed ...
33284113,Pembrolizumab plus axitinib versus sunitinib m...,Background:\nThe first interim analysis of the...


In [20]:
criteria3.to_excel('../../tables/failed_trials/all_phae3_trials_abstract.xlsx')
#criteria3.to_excel('../../tables/fa/otherTumorTypes_criteria3.xlsx')